In [ ]:
import pandas as pd

# Загрузка данных
train_df = pd.read_csv('data/processed/train_data.csv')
test_df = pd.read_csv('data/processed/test_data.csv')

# Проверка загрузки
print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")
print("\nПервые 3 строки train_df:")
print(train_df.head(3))


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
from collections import Counter

def prepare_data(train_df, test_df):
    """Подготовка данных с гарантированным наличием всех классов"""
    # Объединяем редкие классы
    rare_classes = ['NetBIOS', 'LDAP']
    train_df['Label'] = train_df['Label'].replace({k: 'RARE' for k in rare_classes})
    test_df['Label'] = test_df['Label'].replace({k: 'RARE' for k in rare_classes})

    # Фиксируем кодировщик на train данных
    le = LabelEncoder()
    le.fit(train_df['Label'])

    # Кодируем метки
    train_df['Label_encoded'] = le.transform(train_df['Label'])
    test_df['Label_encoded'] = le.transform(test_df['Label'])

    return train_df, test_df, le

def analyze_classes(df, le):
    """Анализ распределения классов"""
    print("\nДетальное распределение:")
    for cls_code, cls_name in enumerate(le.classes_):
        count = sum(df['Label_encoded'] == cls_code)
        print(f"{cls_name} (код {cls_code}): {count} примеров")

def safe_smote_balance(train_df, le, min_samples=50):
    """Безопасная балансировка с проверкой условий"""
    # Выделяем фичи и таргет
    X = train_df.drop(['Label', 'Label_encoded'], axis=1)
    y = train_df['Label_encoded']

    # Анализируем классы перед балансировкой
    analyze_classes(train_df, le)

    # Определяем стратегию увеличения только для минорных классов
    strategy = {}
    for cls in np.unique(y):
        count = sum(y == cls)
        if count < min_samples:
            continue  # Пропускаем слишком малочисленные классы
        target_size = min(count * 3, 50000)  # Увеличиваем не более чем в 3 раза
        strategy[cls] = target_size

    if not strategy:
        print("\nНет классов для балансировки!")
        return train_df

    print("\nСтратегия балансировки:", {le.classes_[k]:v for k,v in strategy.items()})

    # Применяем SMOTE
    try:
        smote = SMOTE(sampling_strategy=strategy, k_neighbors=2)
        X_res, y_res = smote.fit_resample(X, y)
        return pd.concat([pd.DataFrame(X_res), pd.DataFrame(y_res, columns=['Label_encoded'])], axis=1)
    except ValueError as e:
        print(f"\nОшибка балансировки: {e}")
        print("Возвращаем оригинальные данные")
        return train_df

def train_and_validate(train_df, test_df, le):
    """Обучение модели с контролем качества"""
    # Инициализация модели
    task = Task('multiclass', metric='f1_weighted')

    automl = TabularAutoML(
        task=task,
        timeout=3600,
        cpu_limit=4,
        general_params={
            'use_algos': [['lgb', 'cb']],
            'nn_params': {'use_qt': False}
        }
    )

    # Обучение
    roles = {'target': 'Label_encoded'}
    automl.fit_predict(train_df, roles=roles)

    # Проверка наличия всех классов в тесте
    test_classes = set(test_df['Label_encoded'].unique())
    train_classes = set(train_df['Label_encoded'].unique())
    missing_classes = test_classes - train_classes

    if missing_classes:
        print(f"\nПредупреждение: В тесте есть классы, отсутствующие в трейне: {missing_classes}")
        test_df = test_df[~test_df['Label_encoded'].isin(missing_classes)]

    # Предсказание и оценка
    if not test_df.empty:
        test_pred = automl.predict(test_df)
        print("\nОтчёт о классификации:")
        print(classification_report(
            test_df['Label_encoded'],
            test_pred.data.argmax(axis=1),
            target_names=le.classes_,
            digits=4
        ))
    else:
        print("\nТестовые данные не содержат известных классов!")

    return automl

# Главная функция
def main_pipeline(train_df, test_df):
    """Полный пайплайн обработки"""
    # 1. Подготовка данных
    train_df, test_df, le = prepare_data(train_df.copy(), test_df.copy())

    # 2. Балансировка
    balanced_train = safe_smote_balance(train_df, le)

    # 3. Обучение и валидация
    model = train_and_validate(balanced_train, test_df, le)

    return model, le

# Запуск
automl_model, label_encoder = main_pipeline(train_df, test_df)